In [1]:
! pip install pygments

In [3]:
import obspy
import os, glob, sys
import numpy as np
import pandas as pd
from obspy.signal.filter import bandpass
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq
from matplotlib.ticker import LogFormatter

import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.patches as patches

In [4]:
comp = ['ZN', 'ZE', 'NE']
sta_list = np.loadtxt('stalst.redb', dtype='str')
sta_list = sta_list[1:] 
# sta_list = np.loadtxt('stalst.fix', dtype='str')
print(sta_list)


['001' '002' '003' '004' '005' '006a' '006b']


In [5]:
dalst=[]
with open("evtlst.redb", "r") as file:
    for line in file:
        fields = line.split()
        # if fields and int(fields[0]) > 20230601 and int(fields[0]) < 20231110 and (fields[0][7] == "1" or fields[0][7] == "3" or fields[0][7] == "5" or fields[0][7] == "8") :
        if fields and int(fields[0]) > 20230601 :
            dalst.append(fields[0])
print(dalst)

['20230602', '20230603', '20230604', '20230605', '20230606', '20230607', '20230608', '20230609', '20230610', '20230611', '20230612', '20230613', '20230614', '20230615', '20230616', '20230617', '20230618', '20230619', '20230620', '20230621', '20230622', '20230623', '20230624', '20230625', '20230626', '20230627', '20230628', '20230629', '20230630', '20230701', '20230702', '20230703', '20230704', '20230705', '20230706', '20230707', '20230708', '20230709', '20230710', '20230711', '20230712', '20230713', '20230714', '20230715', '20230716', '20230717', '20230718', '20230719', '20230720', '20230721', '20230722', '20230723', '20230724', '20230725', '20230726', '20230727', '20230728', '20230729', '20230730', '20230731', '20230801', '20230802', '20230803', '20230804', '20230805', '20230806', '20230807', '20230808', '20230809', '20230810', '20230811', '20230812', '20230813', '20230814', '20230815', '20230816', '20230817', '20230818', '20230819', '20230820', '20230821', '20230822', '20230823', '20

In [6]:
def plot_WaveSpec(dates, all_waveforms, lagtime, all_spectra, half_freqs, st, comp, target, outfd, suffix):
  # Convert valid_dalst to datetime objects
  dates = pd.to_datetime(valid_dalst, format='%Y%m%d')
  # Determine the min and max values for the color bar

  # Plotting
  fig, ax = plt.subplots(1, 2, figsize=(24, 6), )# gridspec_kw={'width_ratios': [2, 1]})
  fig.suptitle(f"Waveform and Spectrum for {st} - {comp} Component  {target}, {suffix} ", fontsize=16)

  # Plot the waveform using imshow
  c1 = ax[0].imshow(all_waveforms.T, aspect='auto', cmap="RdBu", 
                    extent=[0, len(dates),  lagtime[mask].max(),  lagtime[mask].min()], 
                    vmin=-0.1, vmax=0.1)
                  #   vmin=-max(abs(vmin), abs(vmax))*0.5, vmax=max(abs(vmin), abs(vmax))*0.5)
  fig.colorbar(c1, ax=ax[0], label="Amplitude")


  ax[0].set_title(f"SCFs over time for {st} - {comp} Component")
  ax[0].set_ylabel("Lag Time (s)")

  ax[1].set_title(f"Amplitude Spectrum over time for {st} - {comp} Component")
  ax[1].set_ylabel("Frequency (Hz)")


  # Plot the waveform using imshow
  c2 = ax[1].imshow(all_spectra.T, aspect='auto', cmap="viridis", 
                    extent=[0, len(dates), half_freqs.max(),  half_freqs.min()],
                    # extent=[0, len(dates), 9 , 0],
                    norm=mcolors.LogNorm(vmin=1, vmax=5)) 
  fig.colorbar(c2, ax=ax[1], label="Amplitude " , ticks=[ 0.1,1, 10])

  for pk in range(2):
    ax[pk].set_xticks(np.arange(0, len(dates), 10))  # Set xticks every 10 days
    ax[pk].set_xticklabels(dates[::10].strftime('%Y-%m-%d'), )
    ax[pk].tick_params(axis="x", rotation=90)  # Rotate labels for better readability
    ax[pk].set_xlabel("Date")
    ax[pk].grid(True)
    
  plt.tight_layout()
  plt.savefig(f'{outfd}/Spec_{st}_{cmp}_{suffix}_{target}.png', dpi=200)
  plt.close()

In [7]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

def plot_WaveSpec_all(valid_dalst, all_waveforms, lagtime, all_spectra, half_freqs, st, comp, target, outfd, suffix, coda):
    # Step 1: Convert date strings to datetime
    dates = pd.to_datetime(valid_dalst, format='%Y%m%d')

    # Step 2: Create a full date range from min to max
    full_dates = pd.date_range(start=dates.min(), end=dates.max(), freq='D')

    # Step 3: Build empty arrays for all_waveforms and all_spectra
    n_full_days = len(full_dates)
    n_lags = all_waveforms.shape[1]
    n_freqs = all_spectra.shape[1]

    full_waveforms = np.full((n_full_days, n_lags), np.nan)
    full_spectra = np.full((n_full_days, n_freqs), np.nan)

    # Step 4: Map existing data into full arrays
    date_idx_map = {d: i for i, d in enumerate(full_dates)}
    for i, d in enumerate(dates):
        idx = date_idx_map.get(d)
        if idx is not None:
            full_waveforms[idx, :] = all_waveforms[i, :]
            full_spectra[idx, :] = all_spectra[i, :]

    # Step 5: Plotting
    fig, ax = plt.subplots(1, 2, figsize=(24, 6))
    fig.suptitle(f"Waveform and Spectrum for {st} - {comp} Component  {target}, {suffix}", fontsize=22)
    
    for pk in range(2):
        ax[pk].set_xticks(np.arange(0, n_full_days, 10))
        ax[pk].set_xticklabels(full_dates[::10].strftime('%Y-%m-%d'), rotation=90, fontsize=16)
        ax[pk].set_xlabel("Date", fontsize=20)
        ax[pk].grid(True)
    
    c1 = ax[0].imshow(np.ma.masked_invalid(full_waveforms).T, aspect='auto', cmap="RdBu",
                      extent=[0, n_full_days, lagtime.max(), lagtime.min()],
                      vmin=-0.1, vmax=0.1)
    fig.colorbar(c1, ax=ax[0], ).ax.tick_params(labelsize=16)
    ax[0].set_title(f"SCFs over time for {st} - {comp} Component", fontsize=22)
    ax[0].set_ylabel("Lag Time (s)", fontsize=20)
    
    # Add box from lag time 1–3 seconds across all dates
    rect = patches.Rectangle(
        (0, coda[1]),                      # (x, y) = (start date index, lower lag time)
        n_full_days,                # width spans all dates
        1 - 3,                      # height (Note: y grows downward with imshow)
        linewidth=2,
        edgecolor='black',
        facecolor='none',
        linestyle='--'
    )
    ax[0].add_patch(rect)

    c2 = ax[1].imshow(np.ma.masked_invalid(full_spectra).T, aspect='auto', cmap="viridis",
                      extent=[0, n_full_days, half_freqs.max(), half_freqs.min()],
                      norm=mcolors.LogNorm(vmin=1, vmax=5))
    formatter = LogFormatter(labelOnlyBase=False)
    fig.colorbar(c2, ax=ax[1], ticks=[1,2,3,4,5], format=formatter).ax.tick_params(labelsize=16)
    ax[1].set_title(f"Amplitude Spectrum over time for {st} - {comp} Component", fontsize=22)
    ax[1].set_ylabel("Frequency (Hz)", fontsize=20)
    ax[1].tick_params(labelsize=18)
    ax[0].tick_params(labelsize=18)

    plt.tight_layout()
    # plt.show()
    plt.savefig(f'{outfd}/Specall_{st}_{comp}_{suffix}_{target}.png', dpi=200)
    plt.close()


In [8]:
# sta_list=['005']
listdir = sorted([f for f in glob.glob("COR_sn5_20Hz_F?-?")])
# listdir=['COR_sn5_20Hz_F4-8']
# listdir=['COR_sn5_20Hz_F2-4']

In [18]:
seis_sta=pd.read_csv('redbutte_sta.csv')
seis_sta['stnm']=seis_sta['stnm'].astype(int).apply(lambda i: f"{i:03}")
# seis_sta=seis_sta[:-1] 
seis_sta.loc[6] = seis_sta.loc[5]
seis_sta.at[6, 'stnm'] = '006b'
seis_sta.at[5, 'stnm'] = '006a'
print(seis_sta)

# Mapping dictionary
rename_map = {
    "002": "N051",
    "001": "N037",
    "003": "N018",
    "004": "S009",
    "005": "S024",
    "006a": "S043A",
    "006b": "S043B",
}

# Example: renaming a list of station codes
stations = ["002", "001", "003", "004", "005", "006a","006b"]

renamed_stations = [rename_map[st] for st in stations]
print(renamed_stations)
# Output: ['N037', 'N051', 'N018', 'S009', 'S024', 'S043']
seis_sta['stnm_renamed']=renamed_stations
print(seis_sta)


   stnm        lat         lon    elev
0   002  40.794796 -111.779490  1868.0
1   001  40.794836 -111.779309  1868.0
2   003  40.794740 -111.779110  1847.0
3   004  40.794670 -111.778810  1847.0
4   005  40.794660 -111.778630  1847.0
5  006a  40.794660 -111.778400  1847.0
6  006b  40.794660 -111.778400  1847.0
['N051', 'N037', 'N018', 'S009', 'S024', 'S043A', 'S043B']
   stnm        lat         lon    elev stnm_renamed
0   002  40.794796 -111.779490  1868.0         N051
1   001  40.794836 -111.779309  1868.0         N037
2   003  40.794740 -111.779110  1847.0         N018
3   004  40.794670 -111.778810  1847.0         S009
4   005  40.794660 -111.778630  1847.0         S024
5  006a  40.794660 -111.778400  1847.0        S043A
6  006b  40.794660 -111.778400  1847.0        S043B


In [19]:
outfd='Check_WaveSpectra_renamed'
suffix='ave12-0'
# target='COR_sn5_20Hz_F2-4'

for ndir, target in enumerate(listdir):
    print(target, suffix)
    if ndir ==0:
        coda=[1, 3]  # Define the coda window for the first directory
    else:
        coda=[0.5,2]
    for st in sta_list:
        rename = seis_sta.loc[seis_sta['stnm'] == st, 'stnm_renamed'].values[0]
        for cmp in comp:
            # Storage for amplitude values
            all_amplitudes = []
            all_spectra = []
            valid_dalst = []
            all_waveforms = []
            for da in dalst:
                # print(f"Processing: {da}")

                fd = f'{target}/{st}-{st}'
                # pattern = os.path.join(fd, f'COR_{st}_{st}_{da}.SAC_{cmp}')
                pattern = os.path.join(fd, f'COR_{st}_{st}_{da}.SAC_{cmp}_{suffix}_s')  # Select ZN component
                sacfs = sorted(glob.glob(pattern))

                if not sacfs:
                    # # print(f"Warning: No file found for {da} {target}")
                    continue
                
                valid_dalst.append(da)  # Keep the date if there are corresponding SAC files

                sac = obspy.read(sacfs[0])  # Read SAC file
                ccf = sac[0].data           # Extract cross-correlation function

                sampling_rate = sac[0].stats.sampling_rate
                dt = sac[0].stats.delta
                npts = sac[0].stats.npts
                # print(sacf, 'sampling_rate',sampling_rate, dt, npts)
                
                if npts < 20000:
                    lagtime = np.arange(-100+dt, 100, dt)
                if npts > 20000:
                    lagtime = np.arange(-100-dt, 100, dt)
                if npts == 20000:
                    lagtime = np.arange(-100, 100, dt) 

                # Select the waveform segment between 0-3s
                mask = (lagtime >= 0) & (lagtime <= 6)
                ccf_cut = ccf[mask]
                lag_cut = lagtime[mask]

                all_waveforms.append(ccf_cut)
                    

            # Loop over each waveform to compute its spectrum
            for waveform in all_waveforms:
                # Compute the amplitude of each individual waveform
                N = len(waveform)
                freqs = fftfreq(N, d=dt)  # Frequency axis based on sampling interval dt
                spectrum = np.abs(fft(waveform))  # FFT Amplitude Spectrum

                # Keep only the positive frequencies (first half of the spectrum)
                all_spectra.append(spectrum[:N//2])

                # Store the waveform's amplitude (not averaged)
                all_amplitudes.append(waveform)
                
            # Convert lists to numpy arrays for plotting
            all_waveforms = np.array(all_waveforms)  # Shape: (len(dalst), len(lagtime_cut))
            all_amplitudes = np.array(all_amplitudes)  # Shape: (len(dalst), len(lagtime_cut))
            all_spectra = np.array(all_spectra)  # Shape: (len(dalst), len(freqs//2))
            half_freqs = freqs[:N//2]
            # Convert date strings to numerical indices
            date_indices = np.arange(len(valid_dalst))
            plot_WaveSpec_all(valid_dalst, all_waveforms, lag_cut, all_spectra, half_freqs, rename, cmp, target, outfd, suffix, coda)


        

COR_sn5_20Hz_F2-4 ave12-0
COR_sn5_20Hz_F4-8 ave12-0
